In [10]:
from Bio import SeqIO

In [11]:
genome_file = "../data/ecoli.genbank"
genome_file_type = genome_file.split('.')[-1]

# genome_file = "ecoli.fasta"
# genome_file_type = genome_file.split('.')[-1]

In [12]:
iterator = SeqIO.parse(genome_file, genome_file_type)
record = next(iterator)
length = len(record.seq)
print("read seq from file {}, length = {}".format(genome_file, length))

read seq from file ../data/ecoli.genbank, length = 4641652


In [13]:
import numpy as np
bases_dict     = {"A": 0, "T": 1, "C": 2, "G": 3}
bases_list = ["A", "T", "C", "G"]
bases_np        = {
                    "A": np.array([1, 0, 0, 0], dtype = np.float32),
                    "T": np.array([0, 1, 0, 0], dtype = np.float32),
                    "C": np.array([0, 0, 1, 0], dtype = np.float32),
                    "G": np.array([0, 0, 0, 1], dtype = np.float32)
                    }

seq_np = np.zeros((length, 4), dtype = np.float32) 
for index in range(length):
    base = record.seq[index]
    if base in bases_list:
        channel = bases_dict[base]
        seq_np[index, channel] = 1
    else:
        print("alternative base")

In [14]:
seq_np.shape

(4641652, 4)

In [18]:
LENGTH = 100

def generate_data(seq_np):
    
    length = seq_np.shape[0]
    test = 0.1
    
    train_ids = np.random.choice(int(length * (1-test)) - 300, 10000) + 200
    test_ids = np.random.choice(int(length * test) - 300, 1000) + int(length * (1-test))
    
    print(len(train_ids), min(train_ids), max(train_ids))
    print(len(test_ids), min(test_ids), max(test_ids))
    
    train1 = np.array([seq_np[x-LENGTH:x, ...] for x in train_ids])
    train2 = np.array([seq_np[x+1:x+LENGTH+1, ...] for x in train_ids])
    train_ans = np.array([seq_np[x, ...]  for x in train_ids])

    test1 = np.array([seq_np[x-LENGTH:x, ...] for x in test_ids])
    test2 = np.array([seq_np[x+1:x+LENGTH+1, ...] for x in test_ids])
    test_ans = np.array([seq_np[x, ...]  for x in test_ids])
    
    print(train1.shape, train2.shape, train_ans.shape)
    print( test1.shape, test2.shape, test_ans.shape)
    
    return train1, train2, train_ans, test1, test2, test_ans

In [19]:
# import tensorflow as tf
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [20]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [24]:
import keras
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Activation, Dropout, Flatten, Dense, Input
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.callbacks import EarlyStopping



def build_half_model():
    model = Sequential()
    
    model.add(Conv1D(16, kernel_size=(7), input_shape=(LENGTH, 4)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
#    model.add(MaxPooling1D(pool_size=(3)))
    
#     model.add(Conv1D(4, kernel_size=(3)))
#     model.add(BatchNormalization())
#     model.add(Activation('relu'))
    
    return model

def create_model():
    half1 = build_half_model()
    half2 = build_half_model()

    cur_layer = keras.layers.concatenate([half1.output, half2.output])

    cur_layer = Flatten()(cur_layer)
    cur_layer = Dense(8)(cur_layer)
    cur_layer = Activation('relu')(cur_layer)
    cur_layer = Dropout(0.4)(cur_layer)

    cur_layer = Dense(4, kernel_initializer='normal', activation='sigmoid')(cur_layer)
    
    model = Model([half1.input, half2.input], cur_layer)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

model = create_model()
# model.summary()

In [20]:
model.evaluate([test1, test2], test_ans)

1000/1000 [==============================] - 1s 840us/step


[1.3864956340789796, 0.255]

In [21]:
model.evaluate([train1, train2], train_ans)

10000/10000 [==============================] - 4s 410us/step


[1.3865305728912354, 0.24740000000000001]

In [ ]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', verbose=1, patience=1)
history = model.fit([train1, train2], train_ans, epochs=100, validation_split = 0.1, callbacks = [es])   

Train on 9000 samples, validate on 1000 samples
Epoch 1/100
9000/9000 [==============================] - 10s 1ms/step - loss: 1.3865 - acc: 0.2497 - val_loss: 1.3862 - val_acc: 0.2440
Epoch 2/100
9000/9000 [==============================] - 9s 977us/step - loss: 1.3850 - acc: 0.2542 - val_loss: 1.3860 - val_acc: 0.2420
Epoch 3/100
9000/9000 [==============================] - 9s 1ms/step - loss: 1.3806 - acc: 0.2713 - val_loss: 1.3807 - val_acc: 0.2720
Epoch 4/100
5856/9000 [==================>...........] - ETA: 3s - loss: 1.3774 - acc: 0.2753

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper left')
# plt.show()

# # Plot training & validation loss values
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper left')
# plt.show()

In [ ]:
print(model.evaluate([test1, test2], test_ans))

In [26]:
def train(seq_np):
    train1, train2, train_ans, test1, test2, test_ans = generate_data(seq_np)
    model = create_model()
    es = EarlyStopping(monitor='val_loss', verbose=1, patience=1)
    history = model.fit([train1, train2], train_ans, epochs=100, validation_split = 0.1, callbacks = [es])   
    return model.evaluate([test1, test2], test_ans)[1]

In [27]:
train(seq_np)

10000 374 4177347
1000 4178909 4640090
(10000, 100, 4) (10000, 100, 4) (10000, 4)
(1000, 100, 4) (1000, 100, 4) (1000, 4)
Train on 9000 samples, validate on 1000 samples
Epoch 1/100
9000/9000 [==============================] - 8s 943us/step - loss: 1.3866 - acc: 0.2611 - val_loss: 1.3869 - val_acc: 0.2350
Epoch 2/100
9000/9000 [==============================] - 7s 740us/step - loss: 1.3839 - acc: 0.2711 - val_loss: 1.3875 - val_acc: 0.2340
Epoch 00002: early stopping
1000/1000 [==============================] - 0s 284us/step


0.24099999999999999

In [28]:
scores = [train(seq_np) for i in range(30)]  

10000 218 4177113
1000 4177915 4641248
(10000, 100, 4) (10000, 100, 4) (10000, 4)
(1000, 100, 4) (1000, 100, 4) (1000, 4)
Train on 9000 samples, validate on 1000 samples
Epoch 1/100
9000/9000 [==============================] - 8s 876us/step - loss: 1.3869 - acc: 0.2574 - val_loss: 1.3862 - val_acc: 0.2500
Epoch 2/100
9000/9000 [==============================] - 7s 747us/step - loss: 1.3859 - acc: 0.2580 - val_loss: 1.3862 - val_acc: 0.2510
Epoch 00002: early stopping
1000/1000 [==============================] - 0s 288us/step
10000 653 4177361
1000 4177670 4640923
(10000, 100, 4) (10000, 100, 4) (10000, 4)
(1000, 100, 4) (1000, 100, 4) (1000, 4)
Train on 9000 samples, validate on 1000 samples
Epoch 1/100
9000/9000 [==============================] - 9s 951us/step - loss: 1.3866 - acc: 0.2436 - val_loss: 1.3863 - val_acc: 0.2470
Epoch 2/100
9000/9000 [==============================] - 6s 668us/step - loss: 1.3862 - acc: 0.2489 - val_loss: 1.3861 - val_acc: 0.2600
Epoch 3/100
9000/9000 [==

In [29]:
np.mean(scores)

0.27073333333333333

In [30]:
np.std(scores)

0.020644504245816982

In [31]:
min(scores)

0.23499999999999999

In [32]:
max(scores)

0.312